### FINAL

In [2]:
from csv import reader
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# 초성/중성/종성을 결합하는 함수
def combine_consonants(start, middle, final):

    chosung = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    jungsung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    jongsung = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    
    # 한글 유니코드 계산
    final =  start*588 + middle*28 + final +44032

    return chr(final) 

In [4]:
def draw_Hangul_consonants(path_to_image, path_to_model):

    # 초성, 중성, 종성 세팅 (유니ㅐ코드 인덱스0)
    chosung = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    jungsung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    jongsung = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    # 모델 불러오기
    model = tf.keras.models.load_model(path_to_model)
    
    # 이미지 열기
    image = Image.open(path_to_image)
    image = image.convert('RGB')

    # 이미지 재조정
    image = image.resize((90, 90))

    # 넘파이 배열로 이미지 변환
    image_array = np.asarray(image)
    
    # 픽셀값 표준화
    image_array = [image_array.astype('float32') / 255.0]

    # 리스트를 다시 넘파이 배열로 변환 : 이미지만 변환해서 도출
    X = np.array(image_array)
    
    # 모델 예측
    prediction = model.predict(X)

    max_index_1 = np.argmax(prediction[0][0])
    max_index_2 = np.argmax(prediction[1][0])
    max_index_3 = np.argmax(prediction[2][0])
    
    return combine_consonants(max_index_1, max_index_2, max_index_3)

In [ ]:
# YOLOv8x, ResNet 모델 불러오기
model_YOLO = YOLO('YOLOv8x_best.pt')
model_resnet_path = 'best_ResNet_consonants_model.h5'

# 영상 파일 경로 설정
video_path = "파일경로"

# 영상 불러오기
video = cv2.VideoCapture(video_path)

# 영상 프레임 수와 FPS 가져오기
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# FPS가 0인 경우 1로 설정
if fps == 0:
    fps = 1

# 1초 단위로 이미지 추출하기
for i in range(0, frame_count, fps):
    video.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = video.read()
    if ret:
        # YOLOv8 모델로 객체 탐지
        results = model_YOLO(frame, stream=True)

        # 결과 좌표구하기
        for j, r in enumerate(results):
            boxes = r.boxes.xyxy
            frame_with_boxes = frame.copy()
            
            han_ = []

            # boxes가 비어 있는 경우 루프를 건너뛰기.
            if len(boxes) == 0:
                continue
            
            boxes= sorted(boxes, key=lambda x:x[0])
            
            for box in boxes:
                # YOLOv8 인식 영역 crop
                x1, y1, x2, y2 = [int(b) for b in box]
                cropped_image = frame_with_boxes[y1:y2, x1:x2]
                
                # OpenCV로 Crop된 이미지 선명도 높이기
                alpha = 1.5  
                beta = 30 
                adjusted = cv2.convertScaleAbs(cropped_image, alpha=alpha, beta=beta)

                kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
                emphasized = cv2.filter2D(adjusted, -1, kernel)
                
                alpha1 = 1.0
                dst1 = np.clip((1+alpha1) * emphasized - 128 * alpha1, 0, 255).astype(np.uint8)
                
                dst = cv2.normalize(dst1, None, 0, 255, cv2.NORM_MINMAX)

                cv2.imwrite('A_test.jpg', dst)

                han_.append(draw_Hangul_consonants('A_test.jpg', model_resnet_path))

            print(*han_)